In [3]:
import robotic as ry
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os

# Set the DISPLAY environment variable
os.environ['DISPLAY'] = ':0'

# Initialize configuration
C = ry.Config()
C.addFile("/home/yagz/rai_venv/robotics/Q1/environment.g")
C.view()

# Initialize CameraView for capturing images and depth
cam = ry.CameraView(C)
camera_frames = ["camera1", "camera2", "camera3", "camera4"]
all_points_world = []

# Define the bounding box
bin_min = np.array([-0.3 ,  0.1  ,  0.725])  # Adjusted bounding box
bin_max = np.array([0.3 , 0.7 , 0.77])    # Adjusted bounding box

# Visualize bounding box
f_bbox = C.addFrame("bounding_box", "world")
f_bbox.setShape(ry.ST.ssBox, size=[bin_max[0] - bin_min[0], bin_max[1] - bin_min[1], bin_max[2] - bin_min[2], 0.01])
f_bbox.setPosition([(bin_max[0] + bin_min[0]) / 2, (bin_max[1] + bin_min[1]) / 2, (bin_max[2] + bin_min[2]) / 2])
f_bbox.setColor([0, 1, 0, 0.2])  # Transparent green bounding box
C.view()

# Process each camera
for cam_frame in camera_frames:
    # Set the camera frame
    cam.setCamera(cam_frame)
    
    # Capture RGB and Depth
    rgb, depth = cam.computeImageAndDepth(C)
    depth[depth < 0] = np.nan  # Replace invalid depth values
    pcl = ry.depthImage2PointCloud(depth, cam.getFxycxy())
    pcl = pcl.reshape(-1, 3)  # Flatten to shape (N, 3)

    # Transform point cloud to world frame
    f_cam = C.frame(cam_frame)
    position = f_cam.getPosition()
    quaternion = f_cam.getQuaternion()
    w, x, y, z = quaternion
    rotation_matrix = np.array([
        [1 - 2*y*2 - 2*z*2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x*2 - 2*z*2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x*2 - 2*y*2]
    ])
    pcl_world = (rotation_matrix @ pcl.T).T + position  # Shape (N, 3)

    # Crop point cloud
    mask = np.all((pcl_world >= bin_min) & (pcl_world <= bin_max), axis=1)
    pcl_cropped = pcl_world[mask]

    # Visualize cropped point cloud
    f_crop = C.addFrame(f"crop_pcl_{cam_frame}", "world")
    f_crop.setPointCloud(points=pcl_cropped, colors=[0, 255, 0])
    C.view()
    
    all_points_world.append(pcl_cropped)

# Merge all cropped point clouds
merged_pcl = np.vstack(all_points_world)
print(f"Final merged point cloud shape: {merged_pcl.shape}")

# Visualize merged point cloud
f_pcl = C.addFrame("merged_pcl", "world")
f_pcl.setPointCloud(points=merged_pcl, colors=[0, 0, 255])
C.view()

# --- Estimate normals using Open3D ---
# Step 1: Convert merged point cloud to Open3D PointCloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(merged_pcl)

# Step 2: Estimate normals
# Adjust radius and max_nn for your dataset
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=30))

# Step 3: Ensure normals are consistently oriented
pcd.orient_normals_consistent_tangent_plane(30)

# Step 4: Visualize point cloud with normals
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

# Optional: Convert back to NumPy for further processing
normals = np.asarray(pcd.normals)

# Print a few normals for verification
print(f"Estimated Normals:\n{normals[:5]}")

given point cloud has zero size
given point cloud has zero size
given point cloud has zero size
given point cloud has zero size

-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
Final merged point cloud shape: (0, 3)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.

-- WARNING:frame.cpp:glDr


given point cloud has zero size


RuntimeError: [1;31m[Open3D Error] (void open3d::geometry::PointCloud::OrientNormalsConsistentTangentPlane(size_t)) /home/runner/work/Open3D/Open3D/cpp/open3d/geometry/EstimateNormals.cpp:393: [OrientNormalsConsistentTangentPlane] No normals in the PointCloud. Call EstimateNormals() first.
[0;m

-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying to draw empty mesh (shape type:pointCloud)
-- WARNING:frame.cpp:glDraw:1867(-1) trying

In [4]:
from sklearn.cluster import KMeans
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import robotic as ry
import os

# Set the DISPLAY environment variable
os.environ['DISPLAY'] = ':0'

# Initialize configuration
C = ry.Config()
C.addFile("/home/yagz/rai_venv/robotics/Q1/environment.g")
C.view()

# Initialize CameraView for capturing images and depth
cam = ry.CameraView(C)
camera_frames = ["camera1", "camera2", "camera3", "camera4"]
all_points_world = []

# Define the bounding box
bin_min = np.array([-0.3 ,  0.1  ,  0.725])  # Adjusted bounding box
bin_max = np.array([0.3 , 0.7 , 0.77])    # Adjusted bounding box

# Process each camera
for cam_frame in camera_frames:
    cam.setCamera(cam_frame)
    rgb, depth = cam.computeImageAndDepth(C)
    depth[depth < 0] = np.nan
    pcl = ry.depthImage2PointCloud(depth, cam.getFxycxy()).reshape(-1, 3)

    # Transform point cloud to world frame
    f_cam = C.frame(cam_frame)
    position = f_cam.getPosition()
    quaternion = f_cam.getQuaternion()
    w, x, y, z = quaternion
    rotation_matrix = np.array([
        [1 - 2*y*2 - 2*z*2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x*2 - 2*z*2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x*2 - 2*y*2]
    ])
    pcl_world = (rotation_matrix @ pcl.T).T + position

    # Crop point cloud
    mask = np.all((pcl_world >= bin_min) & (pcl_world <= bin_max), axis=1)
    pcl_cropped = pcl_world[mask]
    all_points_world.append(pcl_cropped)

# Merge all cropped point clouds
merged_pcl = np.vstack(all_points_world)

# Perform K-Means Clustering
n_clusters = 3  # Number of objects in the bin
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
labels = kmeans.fit_predict(merged_pcl)

# Assign colors to clusters
colors = plt.cm.get_cmap("tab10", n_clusters)(labels / (n_clusters - 1))[:, :3]

# Create Open3D point cloud for visualization
o3d_pcl = o3d.geometry.PointCloud()
o3d_pcl.points = o3d.utility.Vector3dVector(merged_pcl)
o3d_pcl.colors = o3d.utility.Vector3dVector(colors)

# Visualize segmented point cloud
o3d.visualization.draw_geometries([o3d_pcl], window_name="Segmented Point Cloud")

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by KMeans.